## Import all necessary packages to notebook

In [97]:
import mat73
import numpy as np
import pandas as pd
import os
import hdf5storage

## get mat files and compile them to excel

In [87]:
##---------------------------------------------------------------
def find_indices(array, freq_bands):
    return {band: (np.argmax(array >= low),
                   np.argmax(array >= high)) for band,
            (low, high) in freq_bands.items()}
##---------------------------------------------------------------
def calculate_mean_for_bands(array, indices_dict):
    band_means = {}
    for band, (start_idx, end_idx) in indices_dict.items():
        band_array = array[start_idx:end_idx+1, :]
        band_mean = np.mean(band_array, axis=0)
        band_means[band] = band_mean
    return band_means

##---------------------------------------------------------------
freq_bands = {'Delta'   :[1, 4],
              'Theta'   :[5, 12],
              'Beta'    :[15, 29],
              'LoGamma' :[30, 49],
              'MidGamma':[51, 99],
              'HiGamma' :[101, 149]}

baseline_period = {'PreInj' :[0, 600]}

chnID_scoreID = {0:'PFC', ##PFCu
                 1:'PFC', ##PFCd
                 2:'dHC',
                 3:'vHC', ##vHCu
                 4:'vHC', ##vHCd
                 5:'MD',
                 6:'CA3'}

meta_cols = ['times','MouseID','ExptDay','DrugID','DrugName']
intervals = 20 # connectivity time in seconds
maxtime   = 3600 # total recording length in secs (60min*60sec)

## Rearrange columns like spectrogram channels arrangement
ChnList  = ['PFC','MD', 'dHC','CA3', 'vHC']    
ChnCmb = []

for froIdx, fromChn in enumerate(ChnList):     
    ChnCmb.append(f'{fromChn}')     

conn_and_freq = meta_cols
for freq in freq_bands.keys():
    for chn in ChnCmb:    
        conn_and_freq.append(f'{chn}_{freq}')
##---------------------------------------------------------

excelPath = 'Path to excel sheet which ha information about electrode scores and Injection details'
drg_table = pd.read_excel(f'{excelPath}Pharma Coh Details.xlsx', sheet_name = 'InjectionDetails')
electode_table = pd.read_excel(f'{excelPath}Pharma Coh Details.xlsx', sheet_name = 'FinalElectrodes')

##---------------------------------------------------------------------------------
conn_list = ['PSD']
dataTypes = []
for df in conn_list:
    for var in ['NoBaseline', 'PreInjBaseline']:
        dataTypes.append(f'{df}_{var}')
        
data_frames = {name: pd.DataFrame() for name in dataTypes}
pltData = {}
writer = pd.ExcelWriter(f'Chronux_PSD connectivity data.xlsx', engine='xlsxwriter') 

### -------------------------------------------------------------------------------------
MatFile_path = 'Path to mat files which need to consolidated'
FileList = [i for i in os.listdir(MatFile_path) if i.endswith('.mat')]

for matFile in FileList:    
    mat  = mat73.loadmat(f'{MatFile_path}{matFile}', ) ## load mat file
    
    MouseID  = int(mat['Ephys']['MouseID'])
    ExptDay  = f"Day{mat['Ephys']['ExptDetails']['Day'][3:]}"
    DrugID   = 'NaN'
    DrugName = 'NaN'
    include  = 'NaN'
    
    for index, row in drg_table.iterrows():
        if row['genoID'] == MouseID and row['Day'] == ExptDay:
            DrugID = row['DrugID']
            DrugName = row['Drug']            
            include = row['Expt_include']
        
    FinalChnIdx = []
    for index, row in electode_table.iterrows():
        if row['genoID'] == MouseID:
            ElectrodeScores = list(row[['PFCu','PFCd','dHC','vHCu','vHCd','MD','CA3']])
            FinalChnIdx = [index  for index, val in enumerate(ElectrodeScores) if val !=0]

    
    freqs = mat['Ephys']['Power']['freqs']
    times = np.arange(intervals, maxtime+intervals, intervals) #start, stop, step
    
    freq_ind = find_indices(freqs, freq_bands)
    bl_ind   = find_indices(times, baseline_period)
    
    if include == 1:
        for con in conn_list:        
            
            ## temporary data frame for data without bl normalization
            temp_df_Nobl = pd.DataFrame()
            temp_df_Nobl['times'] = times
            temp_df_Nobl['MouseID']  = MouseID
            temp_df_Nobl['ExptDay']  = ExptDay
            temp_df_Nobl['DrugID']   = DrugID
            temp_df_Nobl['DrugName'] = DrugName
                     
            ## temporary data frame for bl normalized data
            temp_df_bl = pd.DataFrame()
            temp_df_bl['times'] = times
            temp_df_bl['MouseID']  = MouseID
            temp_df_bl['ExptDay']  = ExptDay
            temp_df_bl['DrugID']   = DrugID
            temp_df_bl['DrugName'] = DrugName
      

            for toChn in FinalChnIdx:                
                   
                chn_combination = f'{chnID_scoreID[toChn]}'
    
                ## Connectivity data without baseline normalization
                connData = mat['Ephys']['Power']['Power'][toChn, :, :] ### [Chnls X Times X Freq] 3D array
                band_means = calculate_mean_for_bands(connData.T, freq_ind) ## calculate mean freq bands (conData should be Freq X times)
    
                for fband in freq_ind.keys():
                    col_name = f'{chn_combination}_{fband}'
                    temp_df_Nobl = pd.concat([temp_df_Nobl, pd.DataFrame(band_means[fband], columns = [col_name])], axis = 1)
    
    
                ##Baseline nomalized data
                baseLine = mat['Ephys']['Power']['Power'][toChn, bl_ind['PreInj'][0]:bl_ind['PreInj'][1]+1, :].mean(axis = 0)
                baseline_array_reshaped = baseLine[np.newaxis, :]
                bl_corrData = (connData/baseline_array_reshaped).T
                # Calculate mean for each frequency band for bl norm data
                band_means = calculate_mean_for_bands(bl_corrData, freq_ind) ###(conData should be Freq X times)
    
                for fband in freq_ind.keys():
                    col_name = f'{chn_combination}_{fband}'
                    temp_df_bl = pd.concat([temp_df_bl, pd.DataFrame(band_means[fband], columns = [col_name])], axis = 1)
            
            ##Concat the temp data to actual df which is stored in dict            
            df2 = data_frames[f'{con}_NoBaseline']            
            NoBaseline_df = pd.concat([df2, temp_df_Nobl.reset_index(drop=True)], axis = 0)
            data_frames[f'{con}_NoBaseline'] = NoBaseline_df
            
            
            df3 = data_frames[f'{con}_PreInjBaseline']            
            PreInjBaseline = pd.concat([df3, temp_df_bl.reset_index(drop=True)], axis = 0)
            data_frames[f'{con}_PreInjBaseline'] = PreInjBaseline
            
            key = f'{DrugName}_{chn_combination}'
                
            # Concatenate baseline-corrected data along the third dimension
            if key not in pltData:
                pltData[key] = {'data': bl_corrData, 'miceIDs': [MouseID]}
            # Add a singleton dimension along axis 2 if needed 
            if pltData[key]['data'].ndim < 3:
                pltData[key]['data'] = pltData[key]['data'][:, :, np.newaxis] 
            else:
                # Concatenate baseline-corrected data along the third dimension
                # Add a singleton dimension to bl_corrData
                pltData[key]['data'] = np.concatenate((pltData[key]['data'], bl_corrData[:, :, np.newaxis]), axis=2)
                pltData[key]['miceIDs'].append(MouseID)

        
        NoBaseLine_df = data_frames[f'{con}_NoBaseline'][conn_and_freq] ## rearrage the channels
        NoBaseLine_df.to_excel(writer, sheet_name=f'{con}_NoBaseline', index=False)
        
        PreInjBaseline_df = data_frames[f'{con}_PreInjBaseline'][conn_and_freq]
        PreInjBaseline_df.to_excel(writer, sheet_name=f'{con}_PreInjBaseline', index=False)

        ##Optional incase want to save mat files to plot the spectrogram
        pltData['freqs'] = freqs
        pltData['times'] = times
        pltData_str_keys = {str(key): value for key, value in pltData.items()}        
        # Specify the file path for saving the .mat file
        mat_file_path = f'PSD_pltData.mat'        
        # Save pltData dictionary as a .mat file
        hdf5storage.savemat(mat_file_path, pltData_str_keys)

writer.close()

## Some electrodes needs to be excluded because of bad placement so use below code to replace excluded electrodes with NaN

In [89]:
def filter_strings_without_substrings(strings_list, substrings_list):
    return list(filter(lambda s: not any(substring in s for substring in substrings_list), strings_list))
##---------------------------------------------------------------------------------------------------------

def filter_strings_with_substrings(strings_list, substrings_list):
    return list(filter(lambda s: any(substring in s for substring in substrings_list), strings_list))
##---------------------------------------------------------------------------------------------------------
## mice list and respective electrodes to be excluded
mouseList = [574, 518]
electList = [['CA3'], ['dHC', 'MD']]

##---------------------------------------------------------------------------------------------------------

writer = pd.ExcelWriter('Chronux_PSD data with final exclusion.xlsx', engine='xlsxwriter')

excelName = 'Chronux_PSD connectivity data.xlsx'
excelPath = 'D:\\GluA1_WT_Pharmacology_analysis\\Power_chronux_20s20s\\'

xls = pd.ExcelFile(f'{excelPath}{excelName}')

# Get the sheet names
sheet_names = xls.sheet_names

for sheet in sheet_names: 
    df = pd.read_excel(f'{excelPath}{excelName}', sheet_name = sheet)
    
    final_df = pd.DataFrame()
    for idx, mouse in enumerate(mouseList):

        with_electrodes = filter_strings_with_substrings(list(df.columns), electList[idx])
        df_with_mouse   = df[df['MouseID'].isin([mouse])]
        df_with_mouse[with_electrodes] = np.nan
        final_df = pd.concat([final_df, df_with_mouse], axis = 0)

    df_without_mouse = df[~df['MouseID'].isin(mouseList)]
    final_df1 = pd.concat([df_without_mouse, final_df], axis = 0)    
    final_df1.to_excel(writer, sheet_name=sheet, index=False)
    
    
writer.close()

C:\Users\dkaetzel\AppData\Local\Temp\ipykernel_19176\4002883782.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_mouse[with_electrodes] = np.nan
C:\Users\dkaetzel\AppData\Local\Temp\ipykernel_19176\4002883782.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_with_mouse[with_electrodes] = np.nan
